In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import re
from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from scipy.spatial.distance import cdist
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import normalize
from wordcloud import WordCloud
from datetime import datetime

In [2]:
tlog = pd.read_csv(r"C:\Users\rtd91\Data\mlops\data\sample_tlog.csv")
product = pd.read_csv(r"C:\Users\rtd91\Data\mlops\data\sample_product.csv")
customer = pd.read_csv(r"C:\Users\rtd91\Data\mlops\data\sample_customer.csv")

In [3]:
def preprocessing(x):  
    ps = PorterStemmer()
    stop_words = set(stopwords.words('english')) 
    filtered_tokenized_text=[]
    tokenized_text = nltk.sent_tokenize(x)
    for sentence in tokenized_text:
        sentence = sentence.lower()
        word_tokens = word_tokenize(sentence)
        filtered_sentence = [ps.stem(w) for w in word_tokens if not w in stop_words]
        filtered_tokenized_text.append(filtered_sentence)
    return "".join([' '.join(text) for text in filtered_tokenized_text])

In [4]:
print(len(tlog["transaction_id"]),tlog["transaction_id"].nunique())
print(len(tlog["customer_id"]),tlog["customer_id"].nunique())
print(len(tlog["upc_no"]),tlog["upc_no"].nunique())

595306 86108
595306 1243
595306 2938


In [5]:
print(len(product["upc_no"]),product["upc_no"].nunique())

2938 2938


In [6]:
print(len(customer["customer_id"]),customer["customer_id"].nunique())

1243 1243


In [9]:
product.head()

,Unnamed: 0,upc_no,upc_desc,category_no_level_1,category_desc_level_1,category_no_level_2,category_desc_level_2
0,0,4600028876,OEP MLD TACO SEASONING MX,2053,MEXICAN SEASONING MIXES,93,TRADITIONAL MEXICAN FOODS
1,1,5210003491,MCCO ORIG TACO SEASNG MIX,2053,MEXICAN SEASONING MIXES,93,TRADITIONAL MEXICAN FOODS
2,2,2100003934,TACOBL ORIG TACO SSNG MIX,2053,MEXICAN SEASONING MIXES,93,TRADITIONAL MEXICAN FOODS
3,3,1111071507,KRO FAJITA SEASONING MIX,2053,MEXICAN SEASONING MIXES,93,TRADITIONAL MEXICAN FOODS
4,4,1111071503,KRO REG TACO SEASONING MX,2053,MEXICAN SEASONING MIXES,93,TRADITIONAL MEXICAN FOODS


In [10]:
customer.head()

,Unnamed: 0,customer_id,convenience_dim_seg,quality_dim_seg,health_dim_seg,price_dim_seg,convenience_dim_score,quality_dim_score,health_dim_score,price_dim_score
0,0,60622933,M,M,M,M,57.237420,58.455336,46.045199,56.256278
1,1,116318566,M,L,M,M,55.043287,36.847263,49.923244,57.525922
2,2,290772182,M,M,M,H,51.416044,46.398561,48.046043,66.436402
3,3,97514036,M,M,M,M,53.418191,57.848860,51.320066,62.298526
4,4,346208265,H,M,L,M,59.661310,57.548050,48.700522,62.308051


In [11]:
tlog.head()

,Unnamed: 0,transaction_id,customer_id,upc_no,category_no_level_1,trans_dt,week_no,store_id,region_cd,purchase_unit,sales_amt,disc_amt,purchase_price
0,0,014_00817_4_121_20190313_18:04:00,14595593,94011,62002,20190313,20190202,817,14,1,1.90,0.0,1.90
1,1,014_00817_3_81_20190417_14:57:00,14595593,94011,62002,20190417,20190303,817,14,1,1.53,0.0,1.53
2,2,014_00817_3_67_20190927_12:51:00,14595593,94011,62002,20190927,20190902,817,14,1,2.03,0.0,2.03
3,3,014_00380_7_68_20190315_13:18:00,238487018,94011,62002,20190315,20190202,380,14,1,2.02,0.0,2.02
4,4,014_00811_525_68_20200131_11:53:00,300620568,94011,62002,20200131,20191304,811,14,1,1.96,0.0,1.96


In [7]:
tlog = pd.merge(tlog, product[["upc_no","upc_desc"]], on='upc_no', how='inner')
tlog
#tlog.join(o","product[["upc_nupc_desc"]],"upc_no","left")

,Unnamed: 0,transaction_id,customer_id,upc_no,category_no_level_1,trans_dt,week_no,store_id,region_cd,purchase_unit,sales_amt,disc_amt,purchase_price,upc_desc
0,0,014_00817_4_121_20190313_18:04:00,14595593,94011,62002,20190313,20190202,817,14,1,1.90,0.00,1.9000,DOLE BANANAS TAPED ORGNC
1,1,014_00817_3_81_20190417_14:57:00,14595593,94011,62002,20190417,20190303,817,14,1,1.53,0.00,1.5300,DOLE BANANAS TAPED ORGNC
2,2,014_00817_3_67_20190927_12:51:00,14595593,94011,62002,20190927,20190902,817,14,1,2.03,0.00,2.0300,DOLE BANANAS TAPED ORGNC
3,3,014_00380_7_68_20190315_13:18:00,238487018,94011,62002,20190315,20190202,380,14,1,2.02,0.00,2.0200,DOLE BANANAS TAPED ORGNC
4,4,014_00811_525_68_20200131_11:53:00,300620568,94011,62002,20200131,20191304,811,14,1,1.96,0.00,1.9600,DOLE BANANAS TAPED ORGNC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595301,595301,014_00466_17_131_20190630_13:39:00,330734391,20799300000,52201,20190630,20190602,466,14,1,20.22,0.00,20.2200,PK BUTT SR 1PC COV
595302,595302,014_00466_10_159_20191006_20:06:00,330734391,20799300000,52201,20191006,20190904,466,14,2,38.09,0.00,19.0450,PK BUTT SR 1PC COV
595303,595303,014_00448_503_43_20190605_08:57:00,116192279,20799300000,52201,20190605,20190502,448,14,1,11.96,6.43,11.9600,PK BUTT SR 1PC COV
595304,595304,014_00355_510_82_20190821_13:54:00,272781162,20799300000,52201,20190821,20190801,355,14,1,9.66,0.00,9.6600,PK BUTT SR 1PC COV


In [13]:
tlog.groupby("week_no").size()

week_no
20190101    10800
20190102    10408
20190103    11922
20190104    11752
20190201    10526
20190202    11496
20190203    10491
20190204    10670
20190301    11677
20190302    10991
20190303    13638
20190304     9652
20190401    11728
20190402    11207
20190403    10918
20190404    11616
20190501    11325
20190502    11449
20190503    11351
20190504    11504
20190601    11115
20190602    12685
20190603    10922
20190604    11432
20190701    10564
20190702    11123
20190703    11589
20190704    11515
20190801    11655
20190802    11792
20190803    12082
20190804    11322
20190901    11343
20190902    10647
20190903    11375
20190904    11194
20191001    11406
20191002    11757
20191003    11291
20191004    11759
20191101    12253
20191102    12083
20191103    12510
20191104    11263
20191201    11566
20191202    12472
20191203    11622
20191204    12034
20191301    11342
20191302    11245
20191303    11655
20191304    11572
dtype: int64

In [9]:
for i in range(len(tlog["week_no"])):
    tlog["week_no"][i] = str(tlog["week_no"][i])#[:4]+"-"+tlog["week_no"].astype(str)[4:6]+"-" + tlog["week_no"][i].astype(str)[6:8]

C:\Users\rtd91\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [10]:
tlog["week_no"] = tlog["week_no"].apply(lambda x:str(x))
tlog["week_no"] = tlog["week_no"].apply(lambda x: x[:4]+"-"+x[4:6]+"-"+x[6:8])

In [11]:
tlog["week_no"] = tlog["week_no"].apply(lambda x: datetime.strptime(x,"%Y-%d-%m"))

In [12]:
results = tlog.groupby(["customer_id","week_no"])["upc_desc"].apply(list)

In [18]:
len(results)

38391

In [13]:
res1= pd.DataFrame(results)

In [14]:
res1

upc_desc
customer_id week_no                                                      
2715888     2019-04-04  [SPKLN ICE CHERRY LIMEADE, KRO APPLES HNYCRISP...
7420205     2019-01-01  [SNYDR FF MINI PRETZELS, SNYDR NBLRS SOURDOUGH...
            2019-01-02  [HTGF CHK ENHC BRST BLSL, LETTUCE BOSTON RPC, ...
            2019-01-04  [PHLLY CREAM CHS BAR 2CT, BROCCOLI CROWNS, PRS...
            2019-01-05  [DRIS STRAWBERRIES PREM, HEINZ TOMATO KETCHUP,...
...                                                                   ...
1881300090  2019-04-13                        [HRSHY MILK CHOCOLATE BARS]
1975400030  2019-02-09                           [KROGER APPLE CIDER NFC]
            2019-02-10  [NSTL SPLSH WILD BERRY 6PK, NSTL SPLSH LEMON 6...
            2019-03-05              [GATORADE FRCE GRAPE, KRO 2% RF MILK]
            2019-03-07  [GATORADE FRCE GRAPE, KRO KICK OF LIME TORT CH...

[38391 rows x 1 columns]

In [21]:
res1.index

MultiIndex([(   2715888, '2019-04-04'),
            (   7420205, '2019-01-01'),
            (   7420205, '2019-01-02'),
            (   7420205, '2019-01-04'),
            (   7420205, '2019-01-05'),
            (   7420205, '2019-01-06'),
            (   7420205, '2019-01-07'),
            (   7420205, '2019-01-08'),
            (   7420205, '2019-01-09'),
            (   7420205, '2019-01-10'),
            ...
            (1881300090, '2019-03-11'),
            (1881300090, '2019-03-12'),
            (1881300090, '2019-04-09'),
            (1881300090, '2019-04-11'),
            (1881300090, '2019-04-12'),
            (1881300090, '2019-04-13'),
            (1975400030, '2019-02-09'),
            (1975400030, '2019-02-10'),
            (1975400030, '2019-03-05'),
            (1975400030, '2019-03-07')],
           names=['customer_id', 'week_no'], length=38391)

In [15]:
customer_id=[]
dates=[]
for i in range(len(res1.index)):
    customer_id.append(res1.index[i][0])
    dates.append(res1.index[i][1])

In [16]:
upc_desc = list(res1["upc_desc"])

In [17]:
df = pd.DataFrame({"customer_id":customer_id, "dates":dates, "upc_desc":upc_desc})

In [18]:
df["num_products"] = df["upc_desc"].apply(lambda x: len(x))

In [64]:
df["flag"] = 0
for i in range(len(df)):
    if ('SPKLN ICE CHERRY LIMEADE' in df["upc_desc"][i] and 'SPKLN ICE PEACH NECTARINE' in df["upc_desc"][i]):
        df["flag"][i] = 1    
    

C:\Users\rtd91\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [66]:
test = df[df["flag"]==1]

In [58]:
df.groupby(["test_product_flag"]).size()

test_product_flag
1    38391
dtype: int64

In [153]:
38391^2

1

In [73]:
def test_model(df):
    max_scores = [] #scores is the number of matches
    products_similarity = {} #customer:scores # inter customer with same customer #intra customer with another customer
    transaction_id=[]
    indice_max_val = []
    index1 = []
    for i in range(len(df)):
        index1.append(i)
        scores = []
        index2 = []
        for j in range(len(df)):
            ## This condition is so that the record of customer is not compared to the same record
            if i == j:
                j=j+1
                continue
            intra_match = 0
            inter_match = 0
                
            #print((set(df["upc_desc"][i]).intersection(set(df["upc_desc"][j]))))
            transaction_id.append(str(i)+"&"+"")
            scores.append((len(set(df["upc_desc"][i]).intersection(set(df["upc_desc"][j])))))
            index2.append(j)
        max_scores.append(max(scores))
        indice_max_val.append(scores.index(max_scores[i]))
        break
    return max_scores, index1, indice_max_val, set(df["upc_desc"][index1][0]) ^ set(df["upc_desc"][indice_max_val][0]) #returns non-intersection items

In [78]:
test_model(df.head(5))

([0], [0], [0], set())